# Reflection
リフレクションは、LLMの生成に続いてリフレクションが行われ、そのリフレクション自体が最初のLLMの出力に基づいて条件付けられた別のLLMの生成を行うデザインパターンです。例えば、コードを書くタスクが与えられた場合、最初のLLMはコードのスニペットを生成し、2番目のLLMはそのコードのスニペットに対する批判を生成します。

AutoGenとエージェントの文脈では、リフレクションはエージェントのペアとして実装できます。最初のエージェントがメッセージを生成し、2番目のエージェントがそのメッセージに対する応答を生成します。2つのエージェントは、最大反復回数や2番目のエージェントの承認など、停止条件に達するまで相互作用を続けます。

AutoGenエージェントを使用して、シンプルな反射デザインパターンを実装してみましょう。

2つのエージェントが存在します：コーダーエージェントとレビュアーエージェントです。コーダーエージェントはコードスニペットを生成し、レビュアーエージェントはコードスニペットの批判を生成します。


<img src="./img/reflection.png" width="400"/>

In [ ]:
#!pip install -U "autogen-agentchat"
#!pip install "autogen-ext[mcp,openai,azure]"

In [ ]:
import logging  
from typing import Any, List  
import os  
from dotenv import load_dotenv  

from autogen_agentchat.agents import AssistantAgent  
from autogen_agentchat.teams import RoundRobinGroupChat  # keeps implementation simple & familiar  
from autogen_agentchat.conditions import TextMessageTermination, MaxMessageTermination, TextMentionTermination, TimeoutTermination

from autogen_core import CancellationToken  
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient  
from autogen_ext.tools.mcp import SseServerParams, mcp_server_tools  
import warnings
warnings.simplefilter('ignore')

load_dotenv()

## OpenTelemetry によるトレーサーのセット
マルチエージェントのデバッグには OpenTelemetry によるトレーサーを利用すると便利。`OpenAIInstrumentor` を使用して OpenAI コールをキャプチャできます。ここではトレース UI として [Jaeger](https://www.jaegertracing.io/download/) を使用しています。

In [ ]:
#!pip install opentelemetry-api opentelemetry-sdk opentelemetry-exporter-otlp opentelemetry-instrumentation-openai

In [ ]:
from opentelemetry import trace
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.instrumentation.openai import OpenAIInstrumentor

service_name = "autogen"

# OTLPエクスポーターの設定 (gRPC経由で送信)
otlp_exporter = OTLPSpanExporter(
    endpoint="http://localhost:4317",  # JaegerのgRPCエンドポイント
)
tracer_provider = TracerProvider(resource=Resource({"service.name": service_name}))
    
# トレーサープロバイダーの設定
trace.set_tracer_provider(tracer_provider)

# バッチスパンプロセッサーを設定
span_processor = BatchSpanProcessor(otlp_exporter)
tracer_provider.add_span_processor(span_processor)

# トレーサーを取得
tracer = tracer_provider.get_tracer(service_name)

OpenAIInstrumentor().instrument()

In [ ]:
mcp_server_uri = os.getenv("MCP_SERVER_URI")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_model = os.getenv("AZURE_OPENAI_MODEL")
azure_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
openai_model_name = os.getenv("OPENAI_MODEL_NAME")

## Tools の定義(MCP over SSE)

In [ ]:

server_params = SseServerParams(  
    url=mcp_server_uri,  
    headers={"Content-Type": "application/json"},  
    timeout=30  
)  

# Fetch tools (async)  
tools = await mcp_server_tools(server_params)  
print(f"Number of Tools: {len(tools)}")
# Set up the OpenAI/Azure model client  
model_client = AzureOpenAIChatCompletionClient(  
    api_key=azure_openai_key,  
    azure_endpoint=azure_openai_endpoint,  
    api_version=api_version,  
    azure_deployment=azure_deployment,  
    model=openai_model_name,  
)  

## エージェント定義

In [ ]:

# 3. -----------------  Agent Definitions -----------------  
primary_agent = AssistantAgent(  
    name="primary",  
    model_client=model_client,  
    tools=tools,  
    description="役立つアシスタント。複数のツールを使用して情報を検索し、質問に回答する",
    system_message=(  
"""
あなたは役立つアシスタントです。複数のツールを使用して情報を検索し、質問に回答することができます。
利用可能なツールを確認し、必要に応じて使用してください。ユーザーが不明な点がある場合は、確認のための質問をすることもできます。
"""
)
)  

critic_agent = AssistantAgent(  
    name="critic",  
    model_client=model_client,  
    description="建設的なフィードバックを提供するデータアナリスト。主に他のエージェントの出力を評価し、改善点を提案する役割を担う。",
    tools=tools,  
    system_message=(
"""
プロのデータアナリストとして建設的なフィードバックを提供してください。フィードバックが反映された場合は 'APPROVE' と回答してください。
"""
    ),  
)  



In [ ]:
# 4. -----------------  Assemble Team -----------------  
participants: List[AssistantAgent] = [  
    primary_agent,  
    critic_agent
]  

In [ ]:
# Define termination condition
max_msg_termination = MaxMessageTermination(max_messages=15)
termination_condition = TextMessageTermination("primary") 
time_terminarion = TimeoutTermination(120)
combined_termination = max_msg_termination | termination_condition | time_terminarion


In [ ]:
# 5. -----------------  Create Team Agent -----------------
team_agent = RoundRobinGroupChat(
    participants=participants,
    termination_condition=combined_termination,
)

In [ ]:
with tracer.start_as_current_span("Reflection") as rollspan: # ルートスパンを作成
    task = "ユーザーID:123 の出荷状況を確認してください。"

    await Console(team_agent.run_stream(task=task))

In [ ]:
with tracer.start_as_current_span("Reflection") as rollspan: # ルートスパンを作成
    #task = "ユーザーID:123 の出荷状況を確認してください。あと頼んだのって何の商品だっけ"
    task = "商品ID:339の商品詳細を教えて"
    await Console(team_agent.run_stream(task=task))

In [ ]:
with tracer.start_as_current_span("Reflection") as rollspan: # ルートスパンを作成
#task = "ユーザーID:123 の出荷状況を確認してください。あと頼んだのって何の商品だっけ"
    task = "SNS分析を行い、日付ごとのツイート数を集計してください"
    await Console(team_agent.run_stream(task=task))

In [ ]:
with tracer.start_as_current_span("Reflection") as rollspan: # ルートスパンを作成
    task = "2024年9月~10月の受注数を集計し、最も受注数が多いかった日付を教えて"
    await Console(team_agent.run_stream(task=task))

In [ ]:
with tracer.start_as_current_span("Reflection") as rollspan: # ルートスパンを作成
    task = "2024年9月~10月の受注数を集計し、さらにSNS分析を行い、日付ごとのツイート数を集計した結果何かわかることはあるか？"
    await Console(team_agent.run_stream(task=task))